In [1]:
import os                                                              
import time   
import pickle
import sys
import numpy as np                                                     
import pandas as pd   
import seaborn as sns                                                  
import matplotlib.pyplot as plt                                        
                                                                       
from sklearn.metrics import mean_absolute_error                        
from sklearn.metrics import mean_squared_error                         
from sklearn.model_selection import train_test_split                   
from sklearn.preprocessing import StandardScaler                       
from sklearn.model_selection import train_test_split                   

import ase                                                             
import ase.build                                                       
from ase import Atoms                                                  
from ase.atoms import Atoms                                            
from ase.io import read, write                                         
from ase.calculators.dftb import Dftb                                  
from ase.units import Hartree, mol, kcal, Bohr                         
                                                                       
from Calculator import src_nogrd                                                       
from Calculator.src_nogrd import sym_func_show                                    
from Calculator.src_nogrd import xyzArr_generator                                 
from Calculator.src_nogrd import feat_scaling_func                                
from Calculator.src_nogrd import at_idx_map_generator
from Calculator.src_nogrd import at_idx_map_generator_old
                                                                                                                           
import pickle
from itertools import combinations_with_replacement as comb_replace
                                                                       
import Utils.DirNav                                                
from Utils.dftb_traj_io import read_scan_traj

In [4]:
root_dir = 'C:\\Users\\Tysh\\Desktop\\'

geom_filename          = os.path.join(root_dir,'Au79.xyz')
md_train_arr_origin    = read_scan_traj(filename=geom_filename)
md_train_arr = md_train_arr_origin.copy(deep=False).reset_index(drop=True)

In [5]:
nAtoms, xyzArr = xyzArr_generator(md_train_arr)# Calculate distance dataframe from xyz coordinates
distances = src_nogrd.distances_from_xyz(xyzArr, nAtoms)

In [6]:
SUPPORTED_ELEMENTS = ['Au', 'O', 'H']

#at_idx_map_naive = at_idx_map_generator_old(md_train_arr[0])
## Hotfix for atom ordering without touching at_idx_map_generator
#at_idx_map_old = {el : at_idx_map_naive[el] for el in SUPPORTED_ELEMENTS}
#at_idx_map = at_idx_map_generator_old(md_train_arr[0])
#print(at_idx_map)

In [8]:
at_idx_map = at_idx_map_generator(md_train_arr[0])
print(at_idx_map)
#print(at_idx_map_naive)

distances.max

KeyError: 'C'